In [2]:
import dlib        
import numpy as np 
import cv2          
import os          
import csv
import dlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')

In [4]:
lip_data=[]
nothing_number=[]
for i in range (5000):
    img = cv2.imread('../datasets/celeba/img/'+str(i)+'.jpg')
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    positions_68_arr = []
    faces = detector(img_gray, 0)
    if len(faces) !=0:
        landmarks = np.matrix([[p.x, p.y] for p in predictor(img, faces[0]).parts()])

        for idx, point in enumerate(landmarks):
        #coordinates of 68 points
            pos = (point[0, 0], point[0, 1])
            positions_68_arr.append(pos)

        positions_lip_arr = []
        #coordinates of 49-68 points which represents mouth
        for i in range(48, 68):
            positions_lip_arr.append(positions_68_arr[i][0])
            positions_lip_arr.append(positions_68_arr[i][1])
        lip_data.append(positions_lip_arr)
    else:
        nothing_number.append(i)
        continue

In [5]:
import pandas as pd
lip_data_dt = pd.DataFrame(lip_data)
lip_data_dt.to_csv('lip.csv')

In [6]:
label=pd.read_table('../datasets/celeba/labels.csv')

In [7]:
# Split the data
x = lip_data
y = label['smiling']

In [8]:
for i in range(len(nothing_number)):
    del y[nothing_number[i]]

# Split the data into training and testing(75% training and 25% testing data)
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=0)# Pre-process data
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [9]:
import pandas as pd
y_dt = pd.DataFrame(y)
y.to_csv('true_label.csv')

In [10]:
clf=SVC(gamma='auto')
clf.fit(x_train,y_train)
y_pred =  clf.predict(x_test)

In [11]:
print('Accuracy Score on train data: ', accuracy_score(y_true=y_train, y_pred=clf.predict(x_train)))
print('Accuracy Score on test data: ', accuracy_score(y_true=y_test, y_pred=y_pred))

Accuracy Score on train data:  0.8976897689768977
Accuracy Score on test data:  0.875515251442704


In [12]:
import pickle

#save model
pickle.dump(clf,open("SVM_smiling_split_dlib_lip.dat","wb"))  

# load model
loaded_model = pickle.load(open("SVM_smiling_split_dlib_lip.dat","rb"))

y_pred=loaded_model.predict(x_test)